In [12]:
from langchain_experimental.llm_symbolic_math.base import LLMSymbolicMathChain
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
from tqdm import tqdm
import pandas as pd
import re
import datasets

def load_model_and_tokenizer(model_name):
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=True)
    model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", use_auth_token=True).to("cuda" if torch.cuda.is_available() else "cpu")
    return model, tokenizer

def create_llm_math_chain(model, tokenizer):
    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_length=512,
        temperature=0,
        top_p=1,
        repetition_penalty=1.0
    )
    llm = HuggingFacePipeline(pipeline=pipe)
    llm_math = LLMSymbolicMathChain.from_llm(llm)
    return llm_math


In [13]:
def react_prompt(question, choices):
    prompt = f"""
You are a highly skilled mathematician tasked with solving complex math problems.

Question: {question}

Choices:
1. {choices[0]}
2. {choices[1]}
3. {choices[2]}
4. {choices[3]}

Let's think step by step:

1. First, identify the type of mathematical problem we are dealing with. 
   - [Here, explain what the problem is asking for in mathematical terms.]

2. Analyze the given choices.
   - [Go through each choice and consider if it could be a potential solution.]

3. Perform necessary calculations or manipulations to arrive at the correct solution.
   - [Show your mathematical work.]

4. Select the correct choice based on the calculations.
   - [Pick the final correct option.]

Final Answer: [State the correct answer clearly as either 1, 2, 3, or 4.]

Explanation: [Provide a brief explanation for why this choice is correct.]
"""
    return prompt


In [14]:
def extract_answer(generated_text):
    answer_pattern = r'Final Answer:\s*(\d)'
    match = re.search(answer_pattern, generated_text, re.IGNORECASE)
    if match:
        answer = int(match.group(1))
        if 1 <= answer <= 4:
            return answer - 1  
    
    last_number_pattern = r'(?:^|\D)([1234])(?:\D|$)'
    matches = list(re.finditer(last_number_pattern, generated_text))
    if matches:
        last_match = matches[-1]
        return int(last_match.group(1)) - 1 
    
    return None


In [15]:
import time

def run_gemma_and_llama_inference(question, choices):
    prompt = react_prompt(question, choices)

    results = {}
    
    for model_name, model_label in model_names.items():
        try:
            start_time = time.time()

            response = client.chat.completions.create(
                model=model_label,
                messages=[{"role": "system", "content": prompt}],
                max_tokens=512,
                temperature=0.7,
                top_p=0.7,
                top_k=50,
                repetition_penalty=1,
                stop=["<eos>", "<end_of_turn>"],
                stream=False
            )

           
            end_time = time.time()
            inference_time = end_time - start_time

            generated_text = response.choices[0].message.content
            extracted_answer = extract_answer(generated_text)

            results[model_name] = {
                "generated_text": generated_text,
                "extracted_answer": extracted_answer + 1 if extracted_answer is not None else None,
                "inference_time": inference_time
            }

        except Exception as e:
            print(f"Error in Together AI model inference for {model_name}: {str(e)}")
            continue

    return results


In [17]:
from datasets import load_dataset

dataset = load_dataset("cais/mmlu", "college_mathematics")
df = pd.DataFrame(dataset['test'])

# Define Together AI model names
model_names = {
    'gemma': 'google/gemma-2b-it',
    'llama': 'meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo'
}

import os
from together import Together
client = Together(api_key="5b9bcdd27be5e30c2f33fbc6b3acf0cbc805c74615418455f0a805f0df9b56a0")  # Replace with your Together API key

results_df = process_dataframe_react_together_gemma_llama(df, model_names)

results_df.to_csv("react_gemma_llama_results.csv", index=False)


  0%|          | 0/100 [00:00<?, ?it/s]

Model: gemma, Question 1:
Extracted answer: 1
Correct answer: 2
Is correct: False
Inference time: 6.8778 seconds



  1%|          | 1/100 [00:09<15:46,  9.56s/it]

Model: llama, Question 1:
Extracted answer: 1
Correct answer: 2
Is correct: False
Inference time: 2.6850 seconds

Model: gemma, Question 2:
Extracted answer: 3
Correct answer: 4
Is correct: False
Inference time: 6.1000 seconds



  2%|▏         | 2/100 [00:18<15:15,  9.34s/it]

...

Model: llama, Question 98:
Extracted answer: 4
Correct answer: 3
Is correct: False
Inference time: 2.5242 seconds

Model: gemma, Question 99:
Extracted answer: 4
Correct answer: 4
Is correct: True
Inference time: 2.8831 seconds



 99%|█████████▉| 99/100 [09:26<00:05,  5.51s/it]

Model: llama, Question 99:
Extracted answer: 2
Correct answer: 4
Is correct: False
Inference time: 1.8745 seconds

Model: gemma, Question 100:
Extracted answer: 3
Correct answer: 1
Is correct: False
Inference time: 3.0585 seconds



100%|██████████| 100/100 [09:32<00:00,  5.72s/it]

Model: llama, Question 100:
Extracted answer: 2
Correct answer: 1
Is correct: False
Inference time: 2.6278 seconds


Accuracy by model:
model
gemma    0.15
llama    0.36
Name: is_correct, dtype: float64


In [18]:
question = "What is the integral of x^2?"
choices = ["x^3/3", "x^2/2", "2x", "1/x"]


results = run_gemma_and_llama_inference(question, choices)

for model_name, result in results.items():
    print(f"Model: {model_name.upper()}")
    print(f"Generated text:\n{result['generated_text']}")
    print(f"Extracted answer: {result['extracted_answer']}")
    print(f"Inference time: {result['inference_time']:.4f} seconds")



Model: GEMMA
Generated text:
**Answer: 1. x^3/3**

**Explanation:**

The integral of x^2 is (x^3)/3. This is a straightforward integral that can be solved by using the power rule of integration.
Extracted answer: 3
Inference time: 0.8958 seconds
Model: LLAMA
Generated text:
Let's break down the problem step by step.

1. First, identify the type of mathematical problem we are dealing with.
   - The problem is asking for the integral of x^2. In mathematical terms, we need to find the antiderivative or the indefinite integral of the function f(x) = x^2.

2. Analyze the given choices.
   - Let's go through each choice and consider if it could be a potential solution:
     1. x^3/3
     2. x^2/2
     3. 2x
     4. 1/x

3. Perform necessary calculations or manipulations to arrive at the correct solution.
   - To find the integral of x^2, we can use the power rule of integration, which states that ∫x^n dx = (x^(n+1))/(n+1) + C, where n is a constant and C is the constant of integration.
   - 